In [ ]:
import pandas as pd
import numpy as np
import gc

# [Experiment] 고차원 특성 공학 (Maximalist Feature Engineering)
# 가설: 가능한 모든 시계열 패턴(Lag, Rolling)과 물리 변수를 생성하면 성능이 극대화될 것이다.
# 목표: 약 110개 이상의 파생 변수 생성

df = pd.read_feather('1차_train.feather')
print("데이터 로드 완료.")

# --- 1. 기본 전처리 ---
df.sort_values(by=['pv_id', 'time'], inplace=True)
g = df.groupby('pv_id')

# --- 2. [핵심] 대규모 시계열 파생 변수 생성 (약 80~90개) ---
print("시차(Lag) 및 이동평균(Rolling) 변수 대량 생성 중...")

# 타겟 변수 확장 (기온, 습도, 구름, 풍속, 기압, 시정, 자외선)
target_cols = ['temp_a', 'humidity', 'cloud_a', 'wind_spd_a', 'pressure', 'vis', 'uv_idx']

# 다양한 시간 윈도우 설정 (데이터 간격이 5분일 경우)
# 3=15분, 6=30분, 12=1시간, 24=2시간, 48=4시간
lags = [3, 6, 12, 24] 
windows = [3, 6, 12, 24] 

# (1) 단순 시차 (Lag) 생성
for col in target_cols:
    for lag in lags:
        df[f'{col}_lag_{lag}'] = g[col].shift(lag)

# (2) 이동 평균 (Rolling Mean) & 표준편차 (Rolling Std) 생성
for col in target_cols:
    for w in windows:
        # 이동 평균 (Trend)
        df[f'{col}_roll_mean_{w}'] = g[col].shift(1).rolling(w).mean()
        # 변동성 (Volatility)
        df[f'{col}_roll_std_{w}'] = g[col].shift(1).rolling(w).std()

# (3) 변화량 (Diff) 생성
for col in target_cols:
    df[f'{col}_diff_1h'] = g[col].diff(12)

# --- 3. 물리적 상호작용 변수 생성 (약 20~30개) ---
print("물리적 상호작용(Physics Interaction) 변수 생성 중...")

# 열역학 및 기상 공식 적용
df['temp_a_sq'] = df['temp_a'] ** 2  # 기온의 비선형성
df['uv_idx_sq'] = df['uv_idx'] ** 2  # 자외선 효과 증폭
df['dew_diff'] = df['temp_a'] - df['dew_point'] # 포화수증기압차(건조도)
df['real_feel_diff'] = df['real_feel_temp'] - df['real_feel_temp_shade'] # 태양 복사열 효과
df['temp_diff_stations'] = df['temp_a'] - df['temp_b'] # 관측소 간 기온 차
df['wind_diff_stations'] = df['wind_spd_a'] - df['wind_spd_b'] # 관측소 간 풍속 차

# 풍력 에너지 (Wind Power density 비례)
df['wind_power_a'] = df['wind_spd_a'] ** 3
df['wind_power_b'] = df['wind_spd_b'] ** 3

# 시간-기상 상호작용 (Cyclical Interaction)
# 시간대별 기상 효과가 다름을 반영 (예: 낮의 구름 vs 밤의 구름)
df['hour'] = df['time'].dt.hour
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24.0)

df['temp_x_time'] = df['temp_a'] * df['hour_cos']
df['cloud_x_time'] = df['cloud_a'] * df['hour_cos']
df['humid_x_time'] = df['humidity'] * df['hour_cos']
df['uv_x_time'] = df['uv_idx'] * df['hour_cos']

# 안개 및 강수 여부 (Binary Flags)
df['is_foggy'] = (df['vis'] < 1).astype(int)
df['is_raining'] = (df['rain'] > 0).astype(int)
df['is_snowing'] = (df['snow'] > 0).astype(int)

# --- 4. 결과 확인 ---
total_cols = len(df.columns)
print(f"특성 공학 완료. 총 컬럼 수: {total_cols} (목표: 110개 이상 달성)")

# [메모리 최적화]
# 생성된 변수가 많아 메모리 부하가 심하므로 Float32 변환 수행
float_cols = df.select_dtypes(include=['float64']).columns
df[float_cols] = df[float_cols].astype('float32')

gc.collect()
print("메모리 최적화(Float32) 완료.")

# [결론]
# 과도한 변수 생성으로 인해 학습 속도가 5배 이상 느려지고, 
# 검증 데이터(Validation)에 대한 과적합(Overfitting) 현상 확인됨.
# 추후 Feature Importance 분석을 통해 상위 50개 변수만 선별(Selection)하기로 결정.